## Imports

In [133]:
import pandas as pd
import yfinance as yf
import numpy as np
import time
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import math
import seaborn as sns
import itertools
import warnings
from tqdm.notebook import tqdm
from itertools import islice

warnings.simplefilter(action='ignore', category=FutureWarning)

## Functions

### Download Data

In [123]:
def download_data(symbols, period, interval, batch_size=100, delay=0.1):
    """
    This function is used to download data for general purposes in a specified format.

    Parameter:
        symbols (list): List of symbols to download data for.
        period (str): Period to download data for. Number + timeframe(d for day, h for hour, m for minute etc.).
        interval (str): Candlesticks timeframe. Same format as period.
        batch_size (int): Size of the batch. Not to overload scanner.
        delay (int): Seconds to wait after each batch. Not to overload scanner.

    Returns:
        dict: Dictionary in format {symbol: symbol_dataframe}.
        Dataframe contains columns: Datetime, Symbol, Open, High, Low, Close, Volume.
    """
    data = {}

    num_batches = (len(symbols) - 1) // batch_size + 1
    for i in tqdm(range(0, len(symbols), batch_size), desc="Downloading batches", total=num_batches):
        batch = symbols[i:i + batch_size]

        df = yf.download(batch, period=period, interval=interval, group_by="ticker", progress=False, threads=True, ignore_tz=True, auto_adjust=False)

        if df.empty:
            continue

        df = pd.concat([df.reset_index()], axis=1)
        df = df.reset_index(drop=True)

        # Rename columns if multi-ticker
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = [
                f"{symbol}_{field}" if field else "Datetime"
                for symbol, field in df.columns
            ]
        else:
            df.columns = [
                f"{batch[0]}_{col}" if col != "Datetime" else "Datetime"
                for col in df.columns
            ]

        # Separate data per ticker
        for symbol in batch:
            selected_columns = [col for col in df.columns if col.startswith(f"{symbol}_")]
            if not selected_columns:
                continue
            symbol_df = df[["Datetime"] + selected_columns].copy()
            symbol_df.columns = [col.split("_", 1)[1] if "_" in col else col for col in symbol_df.columns]
            symbol_df["Symbol"] = symbol
            data[symbol] = symbol_df

        time.sleep(delay)
        
    return data

### Calculate Indicators

In [127]:
def calculate_ema(df, period):
    """
    Calculate a single EMA and return the series.
    Only calculate EMA after enough rows exist (period-1 rows will be NaN).
    """
    ema_series = df["Close"].ewm(span=period, adjust=False).mean()
    ema_series.iloc[:period-1] = pd.NA  # Optional: NaN for initial rows
    return ema_series

def calculate_atr(df, period=14):
    """
    Calculate ATR and ATR%
    """
    high_low = df["High"] - df["Low"]
    high_close = (df["High"] - df["Close"].shift()).abs()
    low_close = (df["Low"] - df["Close"].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df["ATR_14"] = tr.rolling(period).mean()
    df["ATR%"] = df["ATR_14"] / df["Close"] * 100
    return df

def calculate_macd(df):
    """
    Calculate MACD indicators
    """
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Hist"] = df["MACD"] - df["MACD_Signal"]
    return df

def label_candle_color(df):
    """
    Label candles as Green, Red, or Doji
    """
    df["Candle_Color"] = df.apply(
        lambda row: "Green" if row["Close"] > row["Open"]
        else ("Red" if row["Close"] < row["Open"] else "Doji"),
        axis=1
    )
    return df

def mark_crossovers(df, short_ema, long_ema):
    """
    Mark crossovers between two EMA columns.
    True  = short_ema crosses above long_ema
    False = short_ema crosses below long_ema
    NaN   = all other cases
    """
    col = f"EMA_Crossover_{short_ema}_{long_ema}"
    cross_up = (df[f"EMA_{short_ema}"] > df[f"EMA_{long_ema}"]) & (
        df[f"EMA_{short_ema}"].shift(1) <= df[f"EMA_{long_ema}"].shift(1)
    )
    cross_down = (df[f"EMA_{short_ema}"] < df[f"EMA_{long_ema}"]) & (
        df[f"EMA_{short_ema}"].shift(1) >= df[f"EMA_{long_ema}"].shift(1)
    )

    df[col] = pd.Series('No', index=df.index, dtype="object")
    df.loc[cross_up, col] = 'Bullish'
    df.loc[cross_down, col] = 'Bearish'
    return df

def process_symbol_df(symbol_df, ema_periods=[8,20,34,50,200], crossover_emas=[(8, 20)]):
    """
    Round OHLC, force numeric types, and calculate all indicators
    """
    # Ensure numeric
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        if col in symbol_df.columns:
            symbol_df[col] = pd.to_numeric(symbol_df[col], errors='coerce')

    # Round OHLC
    for col in ["Open", "High", "Low", "Close"]:
        symbol_df[col] = symbol_df[col].round(2)

    # Add symbol column
    # symbol_df["Symbol"] = symbol

    # --- Calculate EMAs individually ---
    for period in ema_periods:
        symbol_df[f"EMA_{period}"] = calculate_ema(symbol_df, period)
        symbol_df[f"Above_EMA_{period}"] = symbol_df["Close"] > symbol_df[f"EMA_{period}"]

    # Other indicators
    symbol_df = calculate_atr(symbol_df)
    symbol_df = calculate_macd(symbol_df)
    symbol_df = label_candle_color(symbol_df)
    for ema_1, ema_2 in crossover_emas:
        symbol_df = mark_crossovers(symbol_df, ema_1, ema_2)

    return symbol_df


### Helper Functions

In [92]:
def apply_to_dict(stock_dict, function, **kwargs):
    new_stock_dict = {}
    for sym, df in tqdm(stock_dict.items(), desc="Processing symbols", total=len(stock_dict)):
        df = df.copy()
        df = function(df, **kwargs)
        new_stock_dict[sym] = df
    return new_stock_dict

### Trade Analysis

In [157]:
def generate_trades(df, short_ema: int, long_ema: int):
    trades = []

    # Dynamically pick the crossover column name
    crossover_col = f"EMA_Crossover_{short_ema}_{long_ema}"

    for i in range(1, len(df) - 2):  # leave space for entry day
        prev = df.iloc[i - 1]
        curr = df.iloc[i]
        nxt = df.iloc[i + 1]

        # Detect bullish crossover using precomputed crossover column
        if curr[crossover_col] == 'Bullish' and prev[crossover_col] == 'No':
            entry_day = nxt  # enter next candle
            crossover_day = curr

            # Only enter if new high is made
            if entry_day["High"] > crossover_day["High"]:
                entry_price = crossover_day["High"]
                stop_loss = prev["Low"]  # day before crossover
                risk = entry_price - stop_loss
                pt1 = entry_price + risk
                pt2 = entry_price + 2 * risk

                # Trade outcome flags
                pt1_hit, pt2_hit, stop_loss_hit = False, False, False
                exit_date, exit_price = None, None

                # Track forward until exit
                for j in range(i + 1, len(df)):
                    day = df.iloc[j]

                    # Check PT1
                    if not pt1_hit and day["High"] >= pt1:
                        pt1_hit = True

                    # Check PT2
                    if day["High"] >= pt2:
                        pt2_hit = True
                        exit_date, exit_price = day["Datetime"], pt2
                        break

                    # Check Stop Loss
                    if day["Low"] <= stop_loss:
                        stop_loss_hit = True
                        exit_date, exit_price = day["Datetime"], stop_loss
                        break

                    # Check EMA cross back
                    if day[f"EMA_{short_ema}"] < day[f"EMA_{long_ema}"]:
                        exit_date, exit_price = day["Datetime"], day["Close"]
                        break

                # Crossover metrics
                crossover_vol = crossover_day["Volume"]
                crossover_color = "Bullish" if crossover_day["Close"] > crossover_day["Open"] else "Bearish"

                # Candle color at entry
                candle_color = (
                    "Green" if entry_day["Close"] > entry_day["Open"]
                    else "Red" if entry_day["Close"] < entry_day["Open"]
                    else "Doji"
                )

                # ATR %
                atr_pct = entry_day["ATR_14"] / entry_day["Close"] * 100

                # Append trade
                trades.append({
                    "Datetime": entry_day["Datetime"],
                    "Symbol": df['Symbol'].values[0],
                    "Crossover_Volume": crossover_vol,
                    "Crossover_Color": crossover_color,
                    "Entry Price": entry_price,
                    "Stop Loss": stop_loss,
                    "PT1": pt1,
                    "PT2": pt2,
                    "PT1_Hit": pt1_hit,
                    "PT2_Hit": pt2_hit,
                    "Stop_Loss_Hit": stop_loss_hit,
                    "Exit Date": exit_date,
                    "Exit Price": exit_price,
                })

    trades = pd.DataFrame(trades)
    try:
        trades = pd.merge(trades, df, on=["Datetime", "Symbol"], how="left")
    except Exception as e:
        print(f"Merge error for symbol {df['Symbol'].values[0]}: {e}")

    return trades

## Explore Trades

In [61]:
symbols_data = pd.read_csv('data/all_symbols_results_june_2025.csv', index_col=0).dropna().reset_index(drop=True)
symbols_list = list(symbols_data['Symbol'])

In [81]:
stock_data = download_data(symbols=symbols_list, period='1000d', interval='1d')

Processing batch 1 of 55
Processing batch 2 of 55
Processing batch 3 of 55
Processing batch 4 of 55
Processing batch 5 of 55
Processing batch 6 of 55



2 Failed downloads:
['JNPR', 'X']: YFPricesMissingError('possibly delisted; no price data found  (period=1000d) (Yahoo error = "No data found, symbol may be delisted")')


Processing batch 7 of 55



1 Failed download:
['AZEK']: YFPricesMissingError('possibly delisted; no price data found  (period=1000d) (Yahoo error = "No data found, symbol may be delisted")')


Processing batch 8 of 55
Processing batch 9 of 55
Processing batch 10 of 55
Processing batch 11 of 55


...

Processing batch 53 of 55



1 Failed download:
['WBTN']: Timeout('Failed to perform, curl: (28) Operation timed out after 10002 milliseconds with 588 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Processing batch 54 of 55



1 Failed download:
['BRKL']: Timeout('Failed to perform, curl: (28) Operation timed out after 10001 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Processing batch 55 of 55


In [131]:
stock_data_labeled = apply_to_dict(stock_data, process_symbol_df)

Processing symbols:   0%|          | 0/5443 [00:00<?, ?it/s]

In [158]:
all_trades = {}
for sym, df in tqdm(stock_data_labeled.items(), desc="Generating trades", total=len(stock_data_labeled)):
    trades_df = generate_trades(df, short_ema=8, long_ema=20)
    all_trades[sym] = trades_df

Generating trades:   0%|          | 0/5443 [00:00<?, ?it/s]

Merge error for symbol X: 'Datetime'
Merge error for symbol JNPR: 'Datetime'
Merge error for symbol AZEK: 'Datetime'
Merge error for symbol HMR: 'Datetime'
Merge error for symbol LHSW: 'Datetime'
Merge error for symbol BKHA: 'Datetime'
Merge error for symbol HSPT: 'Datetime'
Merge error for symbol KIDZ: 'Datetime'
Merge error for symbol FSHP: 'Datetime'
Merge error for symbol DIST: 'Datetime'
Merge error for symbol MAMO: 'Datetime'
Merge error for symbol JVSA: 'Datetime'
Merge error for symbol YHNA: 'Datetime'
Merge error for symbol ASPC: 'Datetime'
Merge error for symbol GGR: 'Datetime'
Merge error for symbol EURK: 'Datetime'
Merge error for symbol RDAC: 'Datetime'
Merge error for symbol OCX: 'Datetime'
Merge error for symbol NSPR: 'Datetime'
Merge error for symbol VEEA: 'Datetime'
Merge error for symbol FOSL: 'Datetime'
Merge error for symbol SNCR: 'Datetime'
Merge error for symbol RCT: 'Datetime'
Merge error for symbol LCCC: 'Datetime'
Merge error for symbol RIBB: 'Datetime'
Merge e

In [159]:
concat_entries = pd.concat([df for df in all_trades.values()])

In [160]:
# Additional thresholds 
min_crossover_vol = 2_000_000 # minimum volume at crossover
min_price = 50 # minimum entry price 

# Filter trades combining trend, volatility, price, volume, and candle color 
filtered_combined = concat_entries[ 
    (concat_entries["Above_EMA_34"]) & # trend filter 
    (concat_entries["ATR%"] < concat_entries["ATR%"].median()) & # low volatility 
    (concat_entries["Entry Price"] >= min_price) & # price filter 
    (concat_entries["Crossover_Volume"] >= min_crossover_vol) 
] 

print("Filtered trades:", len(filtered_combined))
print(filtered_combined[["PT1_Hit", "PT2_Hit", "Stop_Loss_Hit"]].mean())

Filtered trades: 2299
PT1_Hit          0.538495
PT2_Hit          0.347977
Stop_Loss_Hit    0.461505
dtype: float64


#### Best filters are:
1. Above 34 EMA
2. Lower volatility: ATR less than 0.035494516184041786
3. Entry Price higher than 50
4. Volume on crossover candle 50

In [538]:
selected_stocks = filtered_combined['Symbol'].unique() # [:50]
selected_stock_data = {key: value for key, value in stock_data_labeled.items() if key in selected_stocks}

## Grid Search of Trading Parameters

### Functions

In [ ]:
def bullish_bearish_pairs(df, short_ema, long_ema):
    col = f'EMA_Crossover_{short_ema}_{long_ema}'
    
    bullish_indices = df.index[df[col] == "Bullish"].tolist()
    bearish_indices = df.index[df[col] == "Bearish"].tolist()

    # Remove first Bearish if it comes before first Bullish
    if bearish_indices and bullish_indices and bearish_indices[0] < bullish_indices[0]:
        bearish_indices.pop(0)

    # Add last index if no Bearish exists after last Bullish
    if bullish_indices and (not bearish_indices or bearish_indices[-1] < bullish_indices[-1]):
        bearish_indices.append(df.index[-1])

    # Pair them one by one
    pairs = list(zip(bullish_indices, bearish_indices))

    # Ensure all pairs are valid (first < second)
    pairs = [(b, r) for b, r in pairs if b < r]

    return pairs

def simulate_trades(df, short_ema, long_ema, stop_rule, sl_offset_pc, max_tp, be_level):
    trades = []

    pairs = bullish_bearish_pairs(df, short_ema, long_ema)
    for bull_cross, bear_cross in pairs:
        # Define trade parameters
        trade = {'Symbol': df['Symbol'].values[0], 'stop_rule': stop_rule, 'offset': sl_offset_pc, 'max_tp': max_tp, 'breakeven': be_level}
        
        # Select days
        pre_cross_day = df.iloc[bull_cross - 1]
        bull_cross_day = df.iloc[bull_cross]
        entry_day = df.iloc[bull_cross + 1] if bull_cross + 1 < len(df) else None

        # Set entry price
        entry_price = bull_cross_day["High"] if entry_day["High"] > bull_cross_day["High"] else None

        # Only enter if new high is made and entry day is not the tail
        if entry_day is None or entry_price is None:
            continue

        # Set stop loss based on rule
        if stop_rule == "crossover":
            stop_loss = bull_cross_day["Low"]
        elif stop_rule == "previous":
            stop_loss = pre_cross_day["Low"]
        else:
            raise ValueError("Invalid stop rule")
        
        # Apply stop loss offset
        stop_loss -= sl_offset_pc / 100 * (entry_price - stop_loss)

        # Filter out trades with invalid stop loss
        if stop_loss >= entry_price:
            continue

        # Calculate risk
        risk = entry_price - stop_loss

        # Set take profit levels, exit sizes and be price
        tp_levels = [entry_price + (i + 1) * risk for i in range(max_tp)]
        # tp_sizes = [1.0 / max_tp] * max_tp  # Position sizes for each TP level
        # tp_sizes = tp_distribution(max_tp, coefficient=tp_coef)  # Position sizes for each TP level
        be_price = entry_price + be_level * risk if be_level > 0 else None

        # Set other trade parameters
        trade['Datetime'] = entry_day["Datetime"]
        trade['EntryPrice'] = entry_price
        trade['StopLoss'] = stop_loss
        trade['Risk'] = risk
        trade['MaxTP'] = tp_levels[-1]
        trade['CrossoverVolume'] = bull_cross_day["Volume"]
        trade['CrossoverColor'] = bull_cross_day["Candle_Color"]
        trade['ExitTypes'] = []
        trade['Exits'] = []
        
        for j in range(bull_cross + 1, bear_cross + 1):
            last_tp_hit = False
            day = df.iloc[j]

            # Check stop loss hit
            if day["Low"] <= stop_loss:
                # Set type based on stop loss relation to entry
                if stop_loss > entry_price:
                    exit_type = 'Trailing_SL'
                elif stop_loss < entry_price:
                    exit_type = 'SL'
                else:
                    exit_type = 'BE'
                trade['ExitTypes'].append(exit_type)
                trade['Exits'].append({'Type': exit_type, 'Date': day["Datetime"], 'Price': stop_loss})
                break

            # Check take profit hits
            for idx, tp in enumerate(tp_levels):
                if day["High"] >= tp:
                    if f'TP{idx+1}' not in trade['ExitTypes']:
                        trade['ExitTypes'].append(f'TP{idx+1}')
                        trade['Exits'].append({'Type': f'TP{idx+1}', 'Date': day["Datetime"], 'Price': tp})
                        stop_loss = tp - 2 * risk if idx >= 1 else be_price
                        # Exit if last TP hit
                        if idx + 1 == max_tp:
                            last_tp_hit = True
                            break  # exit TP loop
            if last_tp_hit:
                break

            if j == bear_cross:
                trade['ExitTypes'].append('EMA_Cross')
                trade['Exits'].append({'Type': 'EMA_Cross', 'Date': day["Datetime"], 'Price': day["Close"]})
                break

        trades.append(trade)
    return trades

In [921]:
def grid_search(df, stop_rules, offsets, tp_levels, be_levels):
    all_trades = []

    # Iterate through all parameter combinations
    for stop_rule, offset, tp_level, be_level in itertools.product(stop_rules, offsets, tp_levels, be_levels):

        # Run simulation
        trades_from_df = simulate_trades(df, 8, 20, stop_rule, offset, tp_level, be_level)

        all_trades.append(trades_from_df)

    return all_trades

In [ ]:
def flatten_all_results(all_results):
    flattened = []

    for sym, sym_trades_out in all_results.items():
        for sym_trades_in in sym_trades_out:
            for trade in sym_trades_in:  # each trade is a dict
                for exit_info in trade["Exits"]:
                    row = {
                        "Symbol": trade["Symbol"],
                        "stop_rule": trade["stop_rule"],
                        "offset": trade["offset"],
                        "max_tp": trade["max_tp"],
                        "breakeven": trade["breakeven"],
                        # "tp_coef": trade["tp_coef"],
                        "Datetime": trade["Datetime"],
                        "EntryPrice": trade["EntryPrice"],
                        "Risk": trade["Risk"],
                        "StopLoss": trade["StopLoss"],
                        "MaxTP": trade["MaxTP"],
                        "CrossoverVolume": trade.get("CrossoverVolume", None),
                        "CrossoverColor": trade.get("CrossoverColor", None),
                        "ExitType": exit_info["Type"],
                        "ExitDate": exit_info["Date"],
                        "ExitPrice": exit_info["Price"],
                        # "ExitSize": exit_info["Size"],
                    }
                    flattened.append(row)

    return pd.DataFrame(flattened)

In [ ]:
def tp_distribution(max_tp, skip_tp=0, coef=1.0):
    """
    Geometric distribution of TP sizes.
    """
    weights = np.array([coef**i for i in range(max_tp-skip_tp)])
    weights = weights / weights.sum()
    weights = [0.0 for i in range(skip_tp)] + weights.tolist()
    return weights

def manage_entries_and_exits(flattened_df, group_cols, skip_tp=0, tp_coef=1.0, entry_dist=None):
    # Get max tp from the dataframe
    max_tp = flattened_df['max_tp'].max()

    df = flattened_df.copy()
    df["ExitSize"] = 0.0

    # Process each trade separately
    for trade_id, group in df.groupby(group_cols):
        max_tp = group["max_tp"].iloc[0]

        # Generate full TP distribution
        tp_sizes = tp_distribution(max_tp, skip_tp, coef=tp_coef)

        # Assign exit sizes
        tp_counter = 0
        for idx, row in group.iterrows():
            exit_size = 0.0
            if row["ExitType"].startswith("TP"):
                tp_idx = int(row["ExitType"][2:]) - 1
                if tp_idx < len(tp_sizes):
                    exit_size = tp_sizes[tp_idx]
                    tp_counter += 1
            elif row["ExitType"] in ["SL", "BE", "Trailing_SL", "EMA_Cross"]:
                # Whatever is left after all TPs
                exit_size = 1 - sum(tp_sizes[:tp_counter])

            df.at[idx, "ExitSize"] = exit_size
            df.at[idx, "tp_coef"] = tp_coef

    return df

def aggregate_trade(group):
    # Weighted average exit price
    weighted_exit_price = (group["ExitPrice"] * group["ExitSize"]).sum()
    
    entry_price = group["EntryPrice"].iloc[0]
    risk = group["Risk"].iloc[0]
    
    # Calculate aggregated results
    return pd.Series({
        "WeightedExitPrice": weighted_exit_price,
        "Return%": (weighted_exit_price - entry_price) / entry_price * 100,
        "R_Multiple": (weighted_exit_price - entry_price) / risk,
        "ExitDate": group["ExitDate"].max(),  # last exit date
        "ExitTypes": ",".join(group["ExitType"].tolist())  # track what exits happened
    })

In [1004]:
def add_indicators_to_trades(trades_df, historical_data_dict):
    enriched_trades = []

    for _, trade in trades_df.iterrows():
        sym = trade["Symbol"]
        dt = trade["Datetime"]

        # start with trade row
        row = trade.to_dict()

        # lookup historical dataframe for this symbol
        if sym in historical_data_dict:
            hist_df = historical_data_dict[sym]
            match = hist_df.loc[hist_df["Datetime"] == dt]

            if not match.empty:
                # take the first row of matching historical data
                hist_data = match.iloc[0].to_dict()

                # merge into trade row
                row.update(hist_data)

        enriched_trades.append(row)

    return pd.DataFrame(enriched_trades)

### Experiments

In [ ]:
# Define parameter grid
stop_rules = ["crossover"] # crossover is the best option
offsets = [0.0, 0.5, 1.0] # not a big impact
tp_levels = [6] # 6 is the best option from 1 to 6, but time in the trade increases
be_levels = [0.5, 1.0] # 1 loos the best so far

# Run grid search across all selected stocks
all_results = {}

for symbol, df in tqdm(selected_stock_data.items(), desc="Grid search by symbol", total=len(selected_stock_data)):
    trades = grid_search(df, stop_rules, offsets, tp_levels, be_levels)
    all_results[df['Symbol'].values[0]] = trades

Grid search by symbol:   0%|          | 0/480 [00:00<?, ?it/s]

In [1045]:
df_exits = flatten_all_results(all_results)

In [1046]:
# Group by identifying columns that define a unique trade
group_cols = [
    "Symbol", "stop_rule", "offset", "max_tp", "breakeven", "Datetime",
    "EntryPrice", "Risk", "StopLoss", "MaxTP", "CrossoverVolume", "CrossoverColor"
]

df_exits = manage_entries_and_exits(df_exits, group_cols, skip_tp=0, tp_coef=1.0)

In [1048]:
tp_coefs = [1.8, 2.0, 2.2] # best values above 2.0, then low impact
position_grid_df = pd.DataFrame()

for tp_coef in tp_coefs:
    df_exits_temp = manage_entries_and_exits(df_exits, group_cols, skip_tp=0, tp_coef=tp_coef)
    position_grid_df = pd.concat([position_grid_df, df_exits_temp], ignore_index=True)

In [1049]:
tp_col = True
group_cols = group_cols + (["tp_coef"] if tp_col else [])

trades_df = position_grid_df.groupby(group_cols).apply(aggregate_trade).reset_index()

In [1050]:
trades_df_historical = add_indicators_to_trades(trades_df, selected_stock_data)

In [1052]:
summary = trades_df_historical.groupby(
    ["stop_rule", "offset", "max_tp", "breakeven", "tp_coef"]
).agg(EV=("R_Multiple", "mean"),
       Winrate=("Return%", lambda x: (x > 0).mean()),
       Trades=("Return%", "count")).reset_index().sort_values(by="EV", ascending=False)

summary.head()

,stop_rule,offset,max_tp,breakeven,tp_coef,EV,Winrate,Trades
11,crossover,0.5,6,1.0,2.2,0.269896,0.533727,7116
17,crossover,1.0,6,1.0,2.2,0.268712,0.534008,7116
10,crossover,0.5,6,1.0,2.0,0.267701,0.533727,7116
5,crossover,0.0,6,1.0,2.2,0.267051,0.533165,7116
16,crossover,1.0,6,1.0,2.0,0.266524,0.534008,7116


In [1053]:
min_crossover_vol = 500_000 # minimum volume at crossover
min_price = 20 # minimum entry price 

# Filter trades combining trend, volatility, price, volume, and candle color 
trades_df_historical_filtered = trades_df_historical[
    (trades_df_historical["Above_EMA_34"]) &
    (trades_df_historical["CrossoverVolume"] >= min_crossover_vol) & 
    (trades_df_historical["Above_EMA_200"]) & 
    (trades_df_historical["Above_EMA_50"]) &
    (trades_df_historical['MACD'] > trades_df_historical['MACD_Signal'])
]

summary_filtered = trades_df_historical_filtered.groupby(
    ["stop_rule", "offset", "max_tp", "breakeven", "tp_coef"]
).agg(EV=("R_Multiple", "mean"),
       Winrate=("Return%", lambda x: (x > 0).mean()),
       Trades=("Return%", "count")).reset_index().sort_values(by="EV", ascending=False)

summary_filtered.head()

,stop_rule,offset,max_tp,breakeven,tp_coef,EV,Winrate,Trades
17,crossover,1.0,6,1.0,2.2,0.445899,0.592780,2770
16,crossover,1.0,6,1.0,2.0,0.442220,0.592780,2770
11,crossover,0.5,6,1.0,2.2,0.442175,0.591336,2770
10,crossover,0.5,6,1.0,2.0,0.438593,0.591336,2770
15,crossover,1.0,6,1.0,1.8,0.437112,0.592780,2770


#### Correlation with EV:
1. Above EMA 34 - highly, positive
2. Crossover Volume - moderate, positive until 500K, negative until 2M, then positive again
3. Price - slightly, negative
4. Above EMA 50 - slightly, positive
5. Above EMA 200 - highly, positive
6. MACD - slightly, positive

*Results from already filtered sample of symbols